In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train.csv.zip')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
###Drop Nan Values
df=df.dropna()

In [5]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [6]:
## Get the Dependent features
y=df['label']

In [7]:
X.shape

(18285, 4)

In [8]:
y.shape

(18285,)

In [9]:
import tensorflow as tf

In [10]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [11]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [12]:
messages=X.copy()

In [13]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [14]:
messages.reset_index(inplace=True)

In [15]:
import nltk
import re
from nltk.corpus import stopwords

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [18]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 

### Embedding Representation

In [19]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2803 1053 2601]
 [   0    0    0 ... 1138 3757  847]
 [   0    0    0 ... 1943 2677 2471]
 ...
 [   0    0    0 ... 1868 1203 2198]
 [   0    0    0 ... 2720 1619 2144]
 [   0    0    0 ... 3378 4485 4178]]


In [20]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  866,
       3925, 2497, 1657, 1854, 4246, 2302, 2803, 1053, 2601])

In [21]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
len(embedded_docs),y.shape

(18285, (18285,))

In [23]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [24]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [26]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10)

Epoch 1/10
383/383 [==============================] - 20s 35ms/step - loss: 0.2772 - accuracy: 0.8731 - val_loss: 0.1981 - val_accuracy: 0.9135
Epoch 2/10
383/383 [==============================] - 13s 35ms/step - loss: 0.1326 - accuracy: 0.9471 - val_loss: 0.1975 - val_accuracy: 0.9181
Epoch 3/10
383/383 [==============================] - 13s 34ms/step - loss: 0.0915 - accuracy: 0.9674 - val_loss: 0.2525 - val_accuracy: 0.9188
Epoch 4/10
383/383 [==============================] - 10s 27ms/step - loss: 0.0620 - accuracy: 0.9781 - val_loss: 0.3153 - val_accuracy: 0.9147
Epoch 5/10
383/383 [==============================] - 10s 27ms/step - loss: 0.0367 - accuracy: 0.9884 - val_loss: 0.3356 - val_accuracy: 0.9183
Epoch 6/10
383/383 [==============================] - 10s 27ms/step - loss: 0.0213 - accuracy: 0.9929 - val_loss: 0.4201 - val_accuracy: 0.9085
Epoch 7/10
383/383 [==============================] - 10s 26ms/step - loss: 0.0132 - accuracy: 0.9956 - val_loss: 0.5615 - val_accuracy:

In [27]:
X_train.shape

(12250, 20)

In [28]:
X_test.shape

(6035, 20)